In [1]:
import os
import json
import pandas as pd

In [2]:
# modificar segun entorno local
os.chdir( "/Volumes/hd_mvf_datasets/henry_data")

# no modificar
folder_data = "1_external"
folder_pipeline = "2_pipeline"
folder_output = "3_output"

In [3]:
# extraemos nombres de cada estado segun nombre de carpeta
state_name = pd.DataFrame(
    pd.DataFrame(
        os.listdir(folder_data)).rename(columns={0:"state"})["state"].str.split("-").str[1])
state_name.head()

,state
0,Indiana
1,Iowa


In [4]:
# valor piloto;
# en el loop seria for i in range(0,max(state_name.index))
i = max(state_name.index)

# creamos una variable con el nombre de estado
# para facilitar el loop para todos los estados
state = state_name["state"][i]
folder_state = "review-" + str(state)

In [5]:
# contamos numero de archivos al interior de cada carpeta del estado
# para facilitar el loop
count_file = 0
# Iterate
for path in os.listdir(os.path.join(folder_data,folder_state)):
    # check if current path is a file
    if os.path.isfile(os.path.join(folder_data,folder_state,path)):
        count_file += 1
count_file

18

In [6]:
# extraemos la informacion
data = []
for f in range(1, count_file):
    with open(
        os.path.join(folder_data,folder_state, str(f) + ".json"), "r") as file:
        for line in file:
            data.append(json.loads(line))
gmaps_state = pd.DataFrame(data)
gmaps_state.info()
gmaps_state.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2550000 entries, 0 to 2549999
Data columns (total 8 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  object
 1   name     object
 2   time     int64 
 3   rating   int64 
 4   text     object
 5   pics     object
 6   resp     object
 7   gmap_id  object
dtypes: int64(2), object(6)
memory usage: 155.6+ MB


,user_id,name,time,rating,text,pics,resp,gmap_id
0,115499682928803397674,Nicole Williams,1528935574441,5,I have found my new favorite nail shop. I love...,None,"{'time': 1528950923185, 'text': 'Thanks'}",0x87ec238afdd8837b:0x2f43f2b12142bff5
1,110072671534171333918,Dmitriy N,1575832576112,5,This place is awesome. Got my pedicure there a...,None,"{'time': 1576094549338, 'text': 'Thanks , Dmit...",0x87ec238afdd8837b:0x2f43f2b12142bff5


In [7]:
# indentificamos las columnas de interes y aquellas que se deben desanidar
vars_interes=[
    "gmap_id" # pk google maps
    ,"user_id"
    ,"name"
    ,"time"
    ,"rating"
    ,"text"
    #,"pics" # irrelevante
    #,"resp" # desanidar
    ]

vars_desanidar=[
    "gmap_id" # pk google maps
    ,"user_id" # pk user >> resp
    ,"resp" # desanidar
    ]


In [8]:
# nos concentramos en: vars_desanidar
gmaps_state_dsndr = pd.DataFrame(gmaps_state[vars_desanidar]).dropna()
gmaps_state_dsndr.info()
gmaps_state_dsndr.head(2)

<class 'pandas.core.frame.DataFrame'>
Index: 257808 entries, 0 to 2549738
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   gmap_id  257808 non-null  object
 1   user_id  257808 non-null  object
 2   resp     257808 non-null  object
dtypes: object(3)
memory usage: 7.9+ MB


,gmap_id,user_id,resp
0,0x87ec238afdd8837b:0x2f43f2b12142bff5,115499682928803397674,"{'time': 1528950923185, 'text': 'Thanks'}"
1,0x87ec238afdd8837b:0x2f43f2b12142bff5,110072671534171333918,"{'time': 1576094549338, 'text': 'Thanks , Dmit..."


In [9]:
# se normaliza la columna
gmaps_state_dsndr = gmaps_state_dsndr.set_index(["gmap_id","user_id"])
gmaps_state_dsndda = pd.json_normalize(
    gmaps_state_dsndr["resp"]).set_index(gmaps_state_dsndr.index)
gmaps_state_dsndda.reset_index(inplace=True) # mueve le indice a una columna
# renombramos para facilitar la union mas abajo
gmaps_state_dsndda.rename(
    columns={"time":"resp_time","text":"resp_text"},
    inplace=True
    )
gmaps_state_dsndda.info()
gmaps_state_dsndda.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257808 entries, 0 to 257807
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   gmap_id    257808 non-null  object
 1   user_id    257808 non-null  object
 2   resp_time  257808 non-null  int64 
 3   resp_text  257808 non-null  object
dtypes: int64(1), object(3)
memory usage: 7.9+ MB


,gmap_id,user_id,resp_time,resp_text
0,0x87ec238afdd8837b:0x2f43f2b12142bff5,115499682928803397674,1528950923185,Thanks
1,0x87ec238afdd8837b:0x2f43f2b12142bff5,110072671534171333918,1576094549338,"Thanks , Dmitriy"


In [10]:
# solo incluimos las variables de interes
gmaps_state_interes = gmaps_state[vars_interes].copy().dropna()
# renombramos para facilitar la union mas abajo
gmaps_state_interes.rename(
    columns={"name":"user_name","time":"user_time","text":"user_text"},
    inplace=True
    )
gmaps_state_interes.info()
gmaps_state_interes.head(2)

<class 'pandas.core.frame.DataFrame'>
Index: 1320011 entries, 0 to 2549878
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   gmap_id    1320011 non-null  object
 1   user_id    1320011 non-null  object
 2   user_name  1320011 non-null  object
 3   user_time  1320011 non-null  int64 
 4   rating     1320011 non-null  int64 
 5   user_text  1320011 non-null  object
dtypes: int64(2), object(4)
memory usage: 70.5+ MB


,gmap_id,user_id,user_name,user_time,rating,user_text
0,0x87ec238afdd8837b:0x2f43f2b12142bff5,115499682928803397674,Nicole Williams,1528935574441,5,I have found my new favorite nail shop. I love...
1,0x87ec238afdd8837b:0x2f43f2b12142bff5,110072671534171333918,Dmitriy N,1575832576112,5,This place is awesome. Got my pedicure there a...


In [11]:
# unir los dataframes
gmaps_state_norm = pd.merge(
    gmaps_state_interes,
    gmaps_state_dsndda,
    on=["gmap_id","user_id"]
    ,how="left"
    )

# generamos una columna state
# para facilitar la union de todos los datos
gmaps_state_norm["state"] = state

# movemos 'state' a la primera fila para facilitar la visualizacion
first_column = gmaps_state_norm.pop("state")
gmaps_state_norm.insert(0, "state", first_column)

# df final por estado
gmaps_state_norm.to_parquet(os.path.join(
    folder_pipeline, str(folder_state) + "_norm.parquet"))
gmaps_state_norm.info()
print(gmaps_state_norm.isna().sum())
gmaps_state_norm.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1327383 entries, 0 to 1327382
Data columns (total 9 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   state      1327383 non-null  object 
 1   gmap_id    1327383 non-null  object 
 2   user_id    1327383 non-null  object 
 3   user_name  1327383 non-null  object 
 4   user_time  1327383 non-null  int64  
 5   rating     1327383 non-null  int64  
 6   user_text  1327383 non-null  object 
 7   resp_time  165590 non-null   float64
 8   resp_text  165590 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 91.1+ MB
state              0
gmap_id            0
user_id            0
user_name          0
user_time          0
rating             0
user_text          0
resp_time    1161793
resp_text    1161793
dtype: int64


,state,gmap_id,user_id,user_name,user_time,rating,user_text,resp_time,resp_text
0,Iowa,0x87ec238afdd8837b:0x2f43f2b12142bff5,115499682928803397674,Nicole Williams,1528935574441,5,I have found my new favorite nail shop. I love...,1.528951e+12,Thanks
1,Iowa,0x87ec238afdd8837b:0x2f43f2b12142bff5,115499682928803397674,Nicole Williams,1528935574441,5,I have found my new favorite nail shop. I love...,1.528951e+12,Thanks
